In [23]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

#creating your first graph and running it

x=tf.Variable(3,name="x")
y=tf.Variable(4,name="y")

f=x*x*y+y+2#creates a computation graph

In [2]:
#--Following code creates a session
sess=tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result=sess.run(f)
print(result)

42


In [3]:
sess.close()

In [4]:
#---here in this way not need to use sess.run() repeatedly
#---and the session automatically closes when the block ends
with tf.Session() as sesss:
    x.initializer.run()
    y.initializer.run()
    result=f.eval()

In [5]:
result

42

In [6]:
#---use global_variables_initializer() function to initialize all
init=tf.global_variables_initializer() #prepare an init node
with tf.Session() as session:
    init.run() #actually initialize all the variables
    result=f.eval()

In [7]:
result

42

In [8]:
#----using Interactive session-----(Inside jupyter or python shell)
# when it is created it automatically set as default
# no with block is needed but you have to close it manually

sess=tf.InteractiveSession()
init.run()
result=f.eval()
#x.graph is tf.get_default_graph()
print(result)

42


In [9]:
sess.close()# closed the session manually

In [10]:
#sess=tf.InteractiveSession()
x1=tf.Variable(1)
x1.graph is tf.get_default_graph()
#sess.close()

True

In [11]:
graph=tf.Graph()
with graph.as_default():
    x2=tf.Variable(2)
x2.graph is graph

True

In [12]:
x2.graph is tf.get_default_graph()

False

In [13]:
# after repeated execution(run) -end up- default graph containing many duplicate blocks
# as a solution , restart jupyter kernel/python shell
# or more conveniently: reset default graph
tf.reset_default_graph()

In [14]:
x2.graph is tf.get_default_graph()#still it is , no raksha

False

In [15]:
import time
w=tf.constant(3)
x=w+2
y=x+5
z=x+3
tic=time.process_time()
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())
toc=time.process_time()
print("computation time:"+str((toc-tic)*1000))

10
8
computation time:12.20359099999957


In [16]:
#calculating z and y, efficiently , without evaluating w and x twice(as usually does)
tic=time.process_time()
with tf.Session() as sess:
    y_val,z_val=sess.run([y,z])
    print(y_val)
    print(z_val)
toc=time.process_time()
print("computation time:"+str((toc-tic)*1000))

10
8
computation time:12.768212000000112


In [24]:
##------LINEAR REGRESSION WITH TENSORFLOW-------####
import numpy as np
from sklearn.datasets import fetch_california_housing

housing=fetch_california_housing()
m,n=housing.data.shape
housing_data_plus_bias=np.c_[np.ones((m,1)),housing.data]
#np.c_ stack 1-D arrays as columns into a 2-D array

X=tf.constant(housing_data_plus_bias,dtype=tf.float32,name='X')
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
XT=tf.transpose(X)
theta=tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)
std_scaler=StandardScaler().fit_transform(X,y)
with tf.Session() as sess:
    theta_value=theta.eval()
    scaled_data=std_scaler.eval()
    #y=y.eval()
    #X=X.eval()
    

ValueError: setting an array element with a sequence.

In [25]:
#-----IMPLEMENTING GRADIENT DESCENT-------

#----Manually Computing Gradients---------

#-------scaling the data set with 

from sklearn.preprocessing import StandardScaler

std_scaler=StandardScaler().fit(X)
scaled_housing_data_plus_bias=std_scaler(housing_data_plus_bias)

n_epochs=1000
learning_rate=0.01

X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")

theta=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name="theta")

y_pred=tf.matmul(X,theta,name="predictions")
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name="mse")
gradients=2/m*tf.matmul(tf.transpose(X),error)
training_op=tf.assign(theta,theta_learning_rate*gradients)

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch%100==0:
            print("Epoch",epoch,"MSE=",mse.eval())
        sess.run(training_op)
    best_theta=theta.eval()

ValueError: setting an array element with a sequence.

In [19]:
help(tf.random_uniform)

Help on function random_uniform in module tensorflow.python.ops.random_ops:

random_uniform(shape, minval=0, maxval=None, dtype=tf.float32, seed=None, name=None)
    Outputs random values from a uniform distribution.
    
    The generated values follow a uniform distribution in the range
    `[minval, maxval)`. The lower bound `minval` is included in the range, while
    the upper bound `maxval` is excluded.
    
    For floats, the default range is `[0, 1)`.  For ints, at least `maxval` must
    be specified explicitly.
    
    In the integer case, the random integers are slightly biased unless
    `maxval - minval` is an exact power of two.  The bias is small for values of
    `maxval - minval` significantly smaller than the range of the output (either
    `2**32` or `2**64`).
    
    Args:
      shape: A 1-D integer Tensor or Python array. The shape of the output tensor.
      minval: A 0-D Tensor or Python value of type `dtype`. The lower bound on the
        range of random val